In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
(xfull,yfull),(xtest,ytest)=keras.datasets.fashion_mnist.load_data()
xvalid,xtrain=xfull[:5000],xfull[5000:]
yvalid,ytrain=yfull[:5000],yfull[5000:]
validset=tf.data.Dataset.from_tensor_slices((xvalid,yvalid))
trainset=tf.data.Dataset.from_tensor_slices((xtrain,ytrain)).shuffle(len(xtrain))
testset=tf.data.Dataset.from_tensor_slices((xtest,ytest))

In [3]:
from tensorflow.train import Feature,Features,Example,BytesList,Int64List
def createexam(image,label):
    imagedata=tf.io.serialize_tensor(image)
    return Example(
        features=Features(
            feature={
                'image':Feature(bytes_list=BytesList(value=[imagedata.numpy()])),
                'label':Feature(int64_list=Int64List(value=[label])),
            }
        )
    )
for image,label in validset.take(1):
    print(createexam(image,int(label)))

features {
  feature {
    key: "image"
    value {
      bytes_list {
        value: "\010\004\022\010\022\002\010\034\022\002\010\034\"\220\006\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\001\000\000\rI\000\000\001\004\000\000\000\000\001\001\000\000\000\000\000\000\000\000\000\000\000\000\000\003\000$\210\177>6\000\000\000\001\003\004\000\000\003\000\000\000\000\000\000\000\000\000\000\000\000\006\000f\314\260\206\220{\027\000\000\000\000\014\n\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\233\354\317\262k\234\241m@\027M\202H\017\000\000\000\000\000\000\000\000\000\000\000\001\000E\317\337\332\330\330\243\177yz\222\215X\254B\000\000\00

In [14]:
from contextlib import ExitStack

def writetfrecode(name,dataset,n_shards=10):
    paths=['{}.tfrecord-{:05d}-of-{:05d}'.format(name,index,n_shards)
         for index in range(n_shards)]
    with ExitStack() as stack:
        writers=[stack.enter_context(tf.io.TFRecordWriter(path))
                for path in paths]
        for index,(image,label)in dataset.enumerate():
            shard=index%n_shards
            example=createexam(image,int(label))
            writers[shard].write(example.SerializeToString())
    return paths
        

In [15]:
trainpath=writetfrecode('myfac.train',trainset)
validpath=writetfrecode('myfac.valid',validset)
testpath=writetfrecode('myfac.test',testset)

In [12]:
from contextlib import ExitStack

def write_tfrecords(name, dataset, n_shards=10):
    paths = ["{}.tfrecord-{:05d}-of-{:05d}".format(name, index, n_shards)
             for index in range(n_shards)]
    with ExitStack() as stack:
        writers = [stack.enter_context(tf.io.TFRecordWriter(path))
                   for path in paths]
        for index, (image, label) in dataset.enumerate():
            shard = index % n_shards
            example = createexam(image, int(label))
            writers[shard].write(example.SerializeToString())
    return paths

In [13]:
train_filepaths = write_tfrecords("my_fashion_mnist.train", trainset)
valid_filepaths = write_tfrecords("my_fashion_mnist.valid", validset)
test_filepaths = write_tfrecords("my_fashion_mnist.test", testset)